In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
from ast import literal_eval
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast, AutoTokenizer
import torch
from collections import OrderedDict
import CXRBERT

from transformers import AutoModelForTokenClassification, AutoModel, AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
RADGRAPH_CLASSES = ['O', 'B-ANAT-DP', 'I-ANAT-DP', 'B-OBS-DP', 'I-OBS-DP', 'B-OBS-DA', 'I-OBS-DA', 'B-OBS-U', 'I-OBS-U']
unique_labels = RADGRAPH_CLASSES
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
print(labels_to_ids)

In [ ]:
new_df = pd.read_csv('/vol/space/users/zengin/ner/train_ner_small.csv', converters={'sentences': literal_eval, 'labels': literal_eval,  'ner_tags': literal_eval})
val_new_df = pd.read_csv('/vol/space/users/zengin/ner/val_ner.csv', converters={'sentences': literal_eval, 'labels': literal_eval,  'ner_tags': literal_eval})
test_new_df = pd.read_csv('/vol/space/users/zengin/ner/test_ner.csv', converters={'sentences': literal_eval, 'labels': literal_eval,  'ner_tags': literal_eval})




tds = Dataset.from_pandas(new_df)
val_tds = Dataset.from_pandas(val_new_df)
test_tds = Dataset.from_pandas(test_new_df)

In [ ]:
model_checkpoint = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code = True)
#model = AutoModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized", trust_remote_code = True)

In [ ]:
## a sanity check for the tokens
inputs = tokenizer(tds[1]["sentences"], is_split_into_words=True)
print(inputs.tokens())
print(inputs.word_ids())

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    new_tags = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label) 
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            
            new_labels.append(label)

    return new_labels

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["sentences"], truncation = True, is_split_into_words=True)
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_inputs = tokenize_and_align_labels(tds)

In [ ]:
train_tokenized_datasets = tds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=tds.column_names,
)
val_tokenized_datasets = val_tds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=val_tds.column_names,
)

test_tokenized_datasets = test_tds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=test_tds.column_names,
)


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
def average_metrics(results_agg):
    results = {}
    for key in results_agg.keys():
        prec = 0
        rec = 0
        f1 = 0
        for key2 in results_agg[key].keys():
            prec += results_agg[key][key2]['precision']
            rec += results_agg[key][key2]['recall']
            f1 += results_agg[key][key2]['f1']
        prec = prec/4
        rec = rec/4
        f1 = f1/4
        results[key] = {'precision': prec, 'recall': rec, 'f1': f1}
    return results
    


In [ ]:

import evaluate
metric = evaluate.load("seqeval")
from seqeval.metrics import classification_report
from nervaluate import Evaluator

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[ids_to_labels[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [ids_to_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels, mode = 'strict', scheme = 'IOB2')
    report = classification_report(true_predictions, true_labels)
    evaluator = Evaluator(true_labels, true_predictions, tags= ['ANAT-DP', 'OBS-DP', 'OBS-DA', 'OBS-U'], loader="list")
    #global results
    #global results_agg
    results, results_agg = evaluator.evaluate()
    results_dict = average_metrics(results_agg)
    
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
        'report': report,
        'muc_results':results_dict
        
        #'nereval_results': results,
        #'nereval_results_agg': results_agg

    }

In [ ]:
#for the pre-trained models
# path = './CXR-BERT/2v0m7nnw/checkpoints/epoch=49-step=25000.ckpt'
# model_cpt = torch.load(path)

# new_state_dict = OrderedDict()
# for k, v in model_cpt['state_dict'].items():
#     # if 'bert' in k:
#     #     name = k[11:] # remove `module.`
#     #     new_state_dict[name] = v
#     # else:
#     name = k[6:] # remove `module.`
#     new_state_dict[name] = v

#model.load_state_dict(new_state_dict)

In [ ]:
from transformers import AutoModelForTokenClassification

model2 = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=ids_to_labels,
    label2id=labels_to_ids,
    trust_remote_code = True,
)

#for pre-trained models in this thesis
# model2.bert.embeddings = model.base_model.embeddings
# model2.bert.encoder = model.base_model.encoder

model2 = model2.to(device)

In [ ]:
# # # ##freezing params for main model and only training the classifier layers
# for param in model2.bert.parameters():
#     param.requires_grad = False
# ###doesnt work it underfits

In [ ]:
#one forward pass to check if the code is working correctly
outputs = model2(input_ids=torch.tensor(train_tokenized_datasets['input_ids'][0]).unsqueeze(0).to(device), labels = torch.tensor(train_tokenized_datasets[0]['labels']).unsqueeze(0).to(device))
outputs



In [ ]:
from transformers import TrainingArguments
metric_name = "precision"
from transformers import EarlyStoppingCallback, IntervalStrategy

args = TrainingArguments(
    "finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none",
    seed = 123,
    metric_for_best_model=metric_name,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model2,
    args=args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)
trainer.train()

In [ ]:
trainer.predict(test_tokenized_datasets)